<a href="https://colab.research.google.com/github/cbadenes/curso-pln/blob/main/notebooks/05_Red_Neuronas_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción a Redes Neuronales con Keras

Este notebook está diseñado para demostrar cómo construir y entrenar un modelo simple de red neuronal utilizando la biblioteca Keras en TensorFlow. Usaremos un conjunto de datos muy básico de frases en español para clasificar su sentimiento como positivo o negativo.

##1) Cargamos y preparamos los datos

El conjunto de datos consiste en frases etiquetadas manualmente para simplificar el uso y la comprensión. Las frases son las siguientes:

- "Me gusta mucho este curso." -> Positivo   
- "Estoy aburrido de la rutina diaria." -> Negativo   
- "El clima hoy es maravilloso." -> Positivo   
- "No estoy satisfecho con el servicio." -> Negativo  

In [21]:
# Example sentences and their labels
sentences = ['Me gusta mucho este curso',
             'Estoy aburrido de la rutina diaria',
             'El clima hoy es maravilloso',
             'No estoy satisfecho con el servicio']
labels = [1, 0, 1, 0]  # 1: Positivo, 0: Negativo

Transformamos el texto en números mediante la técnica ** one-hot encoding** que convierte un texto en una lista de índices enteros, donde cada entero representa una palabra en el texto, codificada mediante una técnica simple de hashing

In [22]:
from tensorflow.keras.preprocessing.text import one_hot

text = "hello world"
vocab_size = 10000

# Codificar el texto
result = one_hot(text, vocab_size)
print(result)


[6310, 296]


Tenemos también que estandarizar la longitud de las secuencias de texto (en este caso, los índices enteros que representan palabras) para que todas tengan el mismo tamaño. Este paso es necesario para entrenar la mayoría de los modelos de aprendizaje profundo, especialmente aquellos que involucran capas que esperan un tamaño de entrada fijo:

In [23]:
encoded_sentences = [[1, 3, 5], [7, 9], [10, 20, 30, 40, 50]]

In [24]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_sentences = pad_sequences(encoded_sentences, maxlen=10, padding='post')
print(padded_sentences)

[[ 1  3  5  0  0  0  0  0  0  0]
 [ 7  9  0  0  0  0  0  0  0  0]
 [10 20 30 40 50  0  0  0  0  0]]


A continuación preparamos los datos de entrenamiento para tener representaciones numéricas del mismo tamaño:

In [33]:
import spacy
import spacy.cli
spacy.cli.download("es_core_news_sm")
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Cargar el modelo de español de Spacy
nlp = spacy.load('es_core_news_sm')

def prepare(corpus, vocab_size=50, max_length=10):
    encoded_sentences = []
    # Crear un diccionario para mapear tokens a índices
    token_to_index = {}
    current_index = 0

    # Primera pasada para construir vocabulario
    for sentence in corpus:
        doc = nlp(sentence)
        for token in doc:
            if token.text not in token_to_index and current_index < vocab_size:
                token_to_index[token.text] = current_index
                current_index += 1

    # Segunda pasada para codificar oraciones
    for sentence in corpus:
        doc = nlp(sentence)
        encoded_sentence = []
        for token in doc:
            # Si el token está en nuestro vocabulario, usar su índice
            if token.text in token_to_index:
                encoded_sentence.append(token_to_index[token.text])
            # Si no está, usar el índice 0 (desconocido)
            else:
                encoded_sentence.append(0)
        encoded_sentences.append(encoded_sentence)

    # Hacer padding de las secuencias
    prepared_sentences = pad_sequences(encoded_sentences, maxlen=max_length, padding='post')
    print("Oraciones procesadas(",len(prepared_sentences),"):")
    print(prepared_sentences)
    print("\nVocabulario (",len(token_to_index),"):")
    print(token_to_index)
    return prepared_sentences, token_to_index

# Uso
vocab_size = 50
max_length = 10
prepared_sentences, vocabulary = prepare(sentences, vocab_size, max_length)

✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Oraciones procesadas( 4 ):
[[ 0  1  2  3  4  0  0  0  0  0]
 [ 5  6  7  8  9 10  0  0  0  0]
 [11 12 13 14 15  0  0  0  0  0]
 [16 17 18 19 20 21  0  0  0  0]]

Vocabulario ( 22 ):
{'Me': 0, 'gusta': 1, 'mucho': 2, 'este': 3, 'curso': 4, 'Estoy': 5, 'aburrido': 6, 'de': 7, 'la': 8, 'rutina': 9, 'diaria': 10, 'El': 11, 'clima': 12, 'hoy': 13, 'es': 14, 'maravilloso': 15, 'No': 16, 'estoy': 17, 'satisfecho': 18, 'con': 19, 'el': 20, 'servicio': 21}


##2) Creamos el modelo basado en una red de neuronas

A continuación, configuraremos un modelo `Sequential`. Este modelo es una pila lineal de capas. Podemos añadir capas con el método `add` y experimentar con diferentes arquitecturas cambiando el número y tipo de capas o ajustando parámetros como el número de neuronas por capa o las funciones de activación.

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
vector_size = 8
model.add(Embedding(vocab_size, vector_size))

# Añadir una capa de aplanado (Flatten) para aplanar la entrada, convirtiendo los datos multidimensionales en un vector unidimensional
model.add(Flatten())

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))

print("Red diseñada correctamente")

Red diseñada correctamente


Compilamos el modelo seleccionando:
- **optimizador (optimizer)**: encargado de cambiar los atributos de la red neuronal como los pesos y la tasa de aprendizaje para reducir las pérdidas (e.g. 'adam', o 'rmsprop')
- **función de pérdida (loss)**: mide como de bien el modelo está haciendo sus predicciones comparadas con los valores reales. El objetivo del entrenamiento es minimizar esta función.
- **evaluación (metrics)**:  métricas utilizadas para evaluar el rendimiento del modelo. No se utilizan para entrenar el modelo pero son importantes para analizar cómo está funcionando el modelo.

In [41]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

##3) Entrenamos el modelo

In [75]:
from sklearn.model_selection import train_test_split
import numpy as np

# Convertir a NumPy arrays para asegurar compatibilidad y rendimiento
prepared_sentences = np.array(prepared_sentences)
labels = np.array(labels)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(prepared_sentences, labels, test_size=0.2, random_state=42)

# Entrenar el modelo
batch_size = 32
epochs = 5
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=epochs)



Epoch 1/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 14s 215ms/step - accuracy: 0.5427 - loss: 0.6872 - val_accuracy: 0.5366 - val_loss: 0.6853
Epoch 2/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 412ms/step - accuracy: 0.6199 - loss: 0.6643 - val_accuracy: 0.5610 - val_loss: 0.6958
Epoch 3/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 19s 360ms/step - accuracy: 0.6198 - loss: 0.6613 - val_accuracy: 0.5610 - val_loss: 0.6820
Epoch 4/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 196ms/step - accuracy: 0.6420 - loss: 0.6411 - val_accuracy: 0.5610 - val_loss: 0.6886
Epoch 5/5
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 362ms/step - accuracy: 0.6287 - loss: 0.6345 - val_accuracy: 0.5610 - val_loss: 0.6923


##4) Evaluamos el modelo

In [78]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Conjunto más amplio de frases de prueba
test_sentences = [
    "no fui al último concierto pq nadie me quería acompañar",
    "Envidio de buena manera a los que tienen la oportunidad de ir mañana al estadio",
    "Se nos está volviendo costumbre del domingo por la noche, ver el episodio anterior de SNL y eso me hace recibir el lunes en un mejor mood",
    "al final decidí no ir al cine porque estaba cansado"
]

# Preparar los datos
prepared_test, vocabulary_test = prepare(test_sentences, vocab_size=vocab_size, max_length=max_length)

# Realizar predicciones
predictions = model.predict(prepared_test)

# Interpretar las predicciones con más detalle
print("Predicciones detalladas:")
for i, sentence in enumerate(test_sentences):
    pred = predictions[i][0]
    sentiment = "Positivo" if pred > 0.5 else "Negativo"
    print(f"\nTexto: {sentence}")
    print(f"Predicción numérica: {pred:.4f}")
    print(f"Sentimiento predicho: {sentiment}")

8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step - accuracy: 0.6009 - loss: 0.6774
Accuracy: 56.10%
Oraciones procesadas( 4 ):
[[ 0  1  2  3  4  5  6  7  8  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [10 11 12 13 14 15 16 17 18 19 11 20 21  2 22  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [23 24 25 26 27 28 29 30 18 31 32 33 34 35 36 11 37 38 39  7 40 41 34 42
  43 44 45 46  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  

# 5) Mejoras

## 5.1) Aumentar el conjunto de entrenamiento

Probemos con el dataset "[cardiffnlp/tweet_sentiment_multilingual](https://huggingface.co/datasets/cardiffnlp/tweet_sentiment_multilingual)" que contiene tweets en varios idiomas, incluido español

In [11]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [34]:
from datasets import load_dataset

# Cargar el dataset desde Hugging Face
dataset = load_dataset("cardiffnlp/tweet_sentiment_multilingual", "spanish", split='train')

# Inspeccionar columnas
print(dataset.column_names)

# Filtrar datos para solo etiquetas positivas y negativas
filtered_data = dataset.filter(lambda x: x['label'] in [0, 2])

# Extraer texto y convertir etiquetas a binario
labels = [1 if label == 2 else 0 for label in filtered_data['label']]
texts = filtered_data['text']
vocab_size = 5000
max_length = 100
prepared_sentences, vocabulary = prepare(texts, vocab_size, max_length)
print(f"Tamaño total del dataset: {len(texts)}")
print(f"Distribución de etiquetas: Positivos={labels.count(1)}, Negativos={labels.count(0)}")

['text', 'label']
Oraciones procesadas( 1226 ):
[[   0    1    2 ...    0    0    0]
 [  12    4   13 ...    0    0    0]
 [  16   17   18 ...    0    0    0]
 ...
 [  33  315   28 ...    0    0    0]
 [ 125  431 1534 ...    0    0    0]
 [  21    0   54 ...    0    0    0]]

Vocabulario ( 5000 ):
{'estoy': 0, 'hasta': 1, 'el': 2, 'ojete': 3, 'de': 4, 'que': 5, 'me': 6, 'digáis': 7, 'tengo': 8, 'cara': 9, 'mala': 10, 'leche': 11, 'Esto': 12, 'estar': 13, 'feliz': 14, 'mola': 15, 'Ya': 16, 'no': 17, 'es': 18, 'tan': 19, 'divertido': 20, '@user': 21, 'con': 22, 'una': 23, 'pequeña': 24, 'donación': 25, 'hará': 26, 'felices': 27, 'a': 28, 'miles': 29, 'chicas': 30, 'tienen': 31, ' ': 32, '#': 33, 'asociacionmariloli': 34, 'He': 35, 'probado': 36, 'nueva': 37, 'espuma': 38, 'para': 39, 'pelo': 40, 'y': 41, 'sí': 42, 'lo': 43, 'deja': 44, 'más': 45, 'rizado': 46, 'pero': 47, 'se': 48, 'queda': 49, 'como': 50, 'efecto': 51, 'gomina': 52, 'gusta': 53, '.': 54, 'aquí': 55, 'tienes': 56, 'mi': 

## 5.3) Añadir más capas

Agregar más capas a un modelo Sequential es sencillo. Basta con usar el método .add() para incluir nuevas capas. A continuación se añade una capa densa adicional y una capa de dropout para regularización. Hay que tener cuidado con no tener sobreajuste:

In [48]:
from tensorflow.keras.layers import Dropout

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
model.add(Embedding(vocab_size, embedding_dim, input_length=100, weights=[embedding_matrix], trainable=False))

#vector_size = 64
#model.add(Embedding(vocab_size, vector_size))

# Añadir una capa de aplanado (Flatten) para aplanar la entrada, convirtiendo los datos multidimensionales en un vector unidimensional
model.add(Flatten())

# Añadimos una nueva capa densa con activación ReLU
model.add(Dense(16, activation='relu'))

# Añadimos una capa de dropout para regularización
model.add(Dropout(0.3))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_6 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

##5.4) Cambiar las funciones de activación

Cambiar la función de activación es tan simple como actualizar el argumento de activación en las capas que lo permitan. Por ejemplo, para cambiar la función de activación de la nueva capa densa a tanh, se puede hacer así:

In [ ]:
# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
model.add(Embedding(vocab_size, embedding_dim, input_length=100, weights=[embedding_matrix], trainable=False))

# Añadir una capa de aplanado (Flatten) para aplanar la entrada, convirtiendo los datos multidimensionales en un vector unidimensional
model.add(Flatten())

# Cambiando a tanh
model.add(Dense(16, activation='tanh'))

# Añadimos una capa de dropout para regularización
model.add(Dropout(0.3))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │          15,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,000 (58.59 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 15,000 (58.59 KB)

##5.5) Utilizar RNN

Para implementar un modelo que utilice redes neuronales recurrentes en Keras, basta con incluir capas como SimpleRNN, LSTM (Long Short-Term Memory), o GRU (Gated Recurrent Units), que son diseñadas para manejar dependencias de secuencias a lo largo del tiempo:

In [60]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
embedding_dim = 128
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
#model.add(Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))

# # SimpleRNN con 64 unidades
#model.add(SimpleRNN(units=64))
model.add(Bidirectional(SimpleRNN(64, return_sequences=False)))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_9 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

###7.1) LSTM

In [74]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
embedding_dim = 128
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
#model.add(Embedding(vocab_size, embedding_dim, input_length=100, weights=[embedding_matrix], trainable=False))

model.add(Bidirectional(LSTM(64, return_sequences=False)))

model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_13 (Embedding)             │ ?                           │       1,500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,500,000 (5.72 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,500,000 (5.72 MB)

###7.2)GRU

In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
embedding_dim = 128
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
#model.add(Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))

# # GRU con 50 unidades
model.add(Bidirectional(GRU(64, return_sequences=False)))

model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

##5.6) Utilizar Embeddings Preentrenados en vez de one-hot encoding

Existen varios embeddings preentrenados disponibles que se podrían utilizar. Algunos de los más populares incluyen:

- Word2Vec: Entrenado en Google News dataset, disponible en varios tamaños.
   
- GloVe (Global Vectors for Word Representation): Disponible en varios tamaños y entrenado en diferentes corpus como Wikipedia o Twitter.

- FastText: Ofrecido por Facebook, similar a Word2Vec pero también considera subpalabras.

###5.6.1) Descarga de embeddings en español

In [66]:
import requests
import gzip
import shutil

# Descargar la versión comprimida de FastText en español (300MB en lugar de varios GB)
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec"
print("Descargando embeddings...")
response = requests.get(url)

# Guardar el archivo
with open("wiki.es.vec", "wb") as f:
    f.write(response.content)

print("Embeddings descargados")

Descargando embeddings...
Embeddings descargados


### 5.6.2) Cargar los Embeddings
Primero, hay que descargar los embeddings y cargarlos en nuestro entorno.

In [67]:
from gensim.models import KeyedVectors
import numpy as np

# Cargamos solo las 10,000 palabras más comunes para ahorrar memoria
embeddings = KeyedVectors.load_word2vec_format('wiki.es.vec', limit=10000)

### 5.6.3) Preparar la Matriz de Embeddings de los datos de entrenamiento
Vamos a crear una matriz de embeddings para usar en la capa de Embedding de Keras. Esta matriz debe tener un vector para cada palabra del vocabulario:

In [73]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer

# Primero creamos un tokenizador
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(sentences)

# Ahora obtenemos el word_index
word_index = tokenizer.word_index

# Crear la matriz de embeddings
embedding_dim = 300  # FastText usa 300 dimensiones
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if i >= vocab_size:
        continue
    try:
        embedding_vector = embeddings[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue

print("Matriz de embeddings creada")

Matriz de embeddings creada


### 5.6.4) Diseñar el Modelo con la Capa de Embedding Preentrenada
Ahora ya podemos crear el modelo utilizando la matriz de embeddings en la capa de Embedding. Es importante establecer trainable=False para no modificar los embeddings durante el entrenamiento:

In [69]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Crear una red secuencial para el modelo
model = Sequential()

# Añadir una capa inicial de embedding que transforma los índices de palabras en vectores densos
model.add(Embedding(vocab_size, embedding_dim, input_length=max_length, weights=[embedding_matrix], trainable=False))

# Añadir una capa de aplanado (Flatten) para aplanar la entrada, convirtiendo los datos multidimensionales en un vector unidimensional
model.add(Flatten())

# Añadir una capa densa con 1 neurona para una salida binaria con una función de activación sigmoid para clasificación binaria
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)             │ ?                           │       1,500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_7 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,500,000 (5.72 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,500,000 (5.72 MB)

A continuación puedes volver al paso 3 para entrenar el modelo